# Zbiór danych

## Niestandardowy zbiór danych w pytorchu

Zakładając że mamy nasze dane w formacie numpy, stworzymy niestandaradowy obiekt pytorch dataset który będzie wczytywał te dane

### Pobranie danych

https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_digits.html#sklearn.datasets.load_digits

In [ ]:
from sklearn.datasets import load_digits

In [ ]:
data = load_digits().data

In [ ]:
data.shape

(1797, 64)

In [ ]:
data[0]

array([ 0.,  0.,  5., 13.,  9.,  1.,  0.,  0.,  0.,  0., 13., 15., 10.,
       15.,  5.,  0.,  0.,  3., 15.,  2.,  0., 11.,  8.,  0.,  0.,  4.,
       12.,  0.,  0.,  8.,  8.,  0.,  0.,  5.,  8.,  0.,  0.,  9.,  8.,
        0.,  0.,  4., 11.,  0.,  1., 12.,  7.,  0.,  0.,  2., 14.,  5.,
       10., 12.,  0.,  0.,  0.,  0.,  6., 13., 10.,  0.,  0.,  0.])

In [ ]:
targets = load_digits().target

In [ ]:
targets.shape

(1797,)

In [ ]:
targets[0]

0

### Podział danych na treningowe i testowe

Do podziały danych na zbiór treningowy i testowy możemy użyć funkcji `train_test_split()` z biblioteki sklearn.

Parametrs `stratify` zapewnie taki sam rozkład danych w  zbiorze treningowym i testowym, w naszym przypadku chcemy aby rozkład klas był taki sam w obu zbiorach więc podajemy `stratify = traget` co zapeni równy rozkład danych względem zmiennej celu


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train_data, test_data, train_targets, test_targets = train_test_split(data, # X
                                                                    targets, # y
                                                                    train_size = 0.8, # cześć danych którą chcemy przeznaczyć na zbiór treningowy
                                                                    stratify = targets # dane według których robimy stratyfikacje
                                                                    )

In [ ]:
train_data.shape

(1437, 64)

In [ ]:
test_data.shape

(360, 64)

### Klasa `Dataset`

Aby stowrzyć niestandardowy dataset w pytrochu musimy utworzyć klasę tego datasetu dziedziczącą po klasie Dataset.

Nasza kalsa musi mieć zaimplementowane conajmiej 3 metody:

`__init__(self, *args, **kwargs)`:
* funkcja wywoływana przy inicjalizacji obiektu, w niej powinniśmy przypisać wszystkie datane używane w datasecie **Uwaga** - najlepiej żeby wszyskie dane były w formacie numpy inaczej dataset może mieć problem z wielowątkowością

`__len__(self):`
* funcja która zwraca rozmiar naszego datasetu

`__getitem__(self, idx)`:
* funkcja która zwraca konkretny element z datasetu i indeksie `idx`


In [ ]:
from torch.utils.data import Dataset

In [ ]:
class DigitsDataset(Dataset):
  def __init__(self, data, targets):
    self.data = data
    self.targets = targets

  def __len__(self):
    return len(self.data)

  def __getitem__(self,idx):
    x = self.data[idx]/16
    y = self.targets[idx]

    return x, y


Inicjalizacja datasetów

In [ ]:
train_dataset = DigitsDataset(train_data, train_targets)

In [ ]:
test_dataset = DigitsDataset(test_data, test_targets)

In [ ]:
single_item =  test_dataset[0]

x

In [ ]:
single_item[0]

array([0.    , 0.    , 0.1875, 0.5   , 0.6875, 0.6875, 0.0625, 0.    ,
       0.    , 0.    , 0.1875, 1.    , 1.    , 0.75  , 0.    , 0.    ,
       0.    , 0.    , 0.125 , 0.9375, 1.    , 0.75  , 0.    , 0.    ,
       0.    , 0.    , 0.    , 1.    , 1.    , 0.4375, 0.    , 0.    ,
       0.    , 0.    , 0.0625, 0.9375, 1.    , 0.625 , 0.    , 0.    ,
       0.    , 0.    , 0.0625, 1.    , 1.    , 0.375 , 0.    , 0.    ,
       0.    , 0.    , 0.1875, 1.    , 1.    , 0.3125, 0.    , 0.    ,
       0.    , 0.    , 0.125 , 0.9375, 1.    , 0.375 , 0.    , 0.    ])

y

In [ ]:
single_item[1]

1

## Dataloader

`Dataloader` jest podiewdzialny za wczytywanie danych z datasetu - dzielenia ich na paczki (batch) które są przetwarzanie jednoczeście, wczytywanie ich w określonej kolejności itd.

In [ ]:
from torch.utils.data import DataLoader

In [ ]:
train_dataloader = DataLoader(train_dataset, batch_size = 32, shuffle = True)

In [ ]:
test_dataloader = DataLoader(test_dataset, batch_size  = 32, shuffle = False)

Wczytanie pojedyńczego batcha

In [ ]:
single_batch = next(iter(test_dataloader))

x

In [ ]:
single_batch[0].shape

torch.Size([32, 64])

In [ ]:
single_batch[0].max()

tensor(1., dtype=torch.float64)

y

In [ ]:
single_batch[1].shape

torch.Size([32])

# Sieć neuronowa

## Warsty w pytorchu

Postawowe warstwy dostępne w pytorchu możemy znaleść w `torch.nn`

Warstwa fully connected : `nn.Linear(in_features, out_features)`
* in_features - rozmiar inputu warstwy
* out_features - rozmiar outputu warsty
* *Przypomienie: wagi w warswie to macierz (out_features, in_features)*

In [ ]:
from torch import nn

In [ ]:
fc = nn.Linear(10,20)

In [ ]:
fc

Linear(in_features=10, out_features=20, bias=True)

Na początku wagi są losowe

In [ ]:
fc.weight

Parameter containing:
tensor([[ 0.1398, -0.0514,  0.2011,  0.1314, -0.0848,  0.2049,  0.1839,  0.1968,
          0.1566, -0.2067],
        [ 0.0752, -0.0343,  0.1289,  0.1007, -0.2857, -0.3008, -0.3083, -0.2215,
          0.0375, -0.1452],
        [ 0.0080,  0.1400, -0.2615, -0.1307, -0.2812, -0.2015, -0.1892, -0.0938,
          0.1482,  0.1900],
        [-0.2705, -0.2064, -0.2946, -0.0136, -0.1918, -0.2502,  0.1915, -0.0381,
         -0.1822,  0.1204],
        [ 0.1501,  0.0034,  0.2133,  0.2295,  0.1062,  0.1396,  0.1197, -0.0951,
         -0.0885, -0.1080],
        [-0.0863, -0.1228,  0.1305,  0.1781,  0.0020,  0.0913,  0.1370, -0.2568,
          0.2314,  0.1465],
        [ 0.0825, -0.0892,  0.2579,  0.1013, -0.2274, -0.1461,  0.0509,  0.2408,
         -0.1753, -0.2798],
        [ 0.2586, -0.2380,  0.1515, -0.0184,  0.2930,  0.0832, -0.0728, -0.2953,
          0.2620,  0.1674],
        [ 0.2547,  0.1471, -0.2072,  0.0425,  0.0465,  0.1266,  0.0639,  0.1853,
          0.0463, -0.1660

In [ ]:
fc.weight.shape

torch.Size([20, 10])

## Funkcje aktywacji w pytorchu

Podstawowe fukncje aktywacji dostępne są w torch.nn.functional

In [ ]:
import torch.nn.functional as F

In [ ]:
from torch import Tensor

In [ ]:
F.softmax(Tensor([1,2,3]), dim=0)

tensor([0.0900, 0.2447, 0.6652])

## Sieć neuronowa

Aby stowrzyć sieć neuronową w pytorchu musimy stworzyć klasę dziedziczącą po klasie `torch.nn.Module`

Nasza klasa musi mieć zaimplementowane conajmniej 2 funkcje:

`__init__(self, *args, **kwargs)`
* inicjalizuje wszyskie parametry sieci (np. warstwy)

`forward(self, x)`
* implementuje logikę przekazywania danych w przód sieci neuronowej

In [ ]:
class Model(nn.Module):
  def __init__(self, input_size, num_classes):
    super(Model, self).__init__()
    self.fc1 = nn.Linear(input_size, 100) # tworzymy pierwszą warstwę
    self.fc2 = nn.Linear(100, num_classes)  # tworzymy drugą warstwę

  def forward(self, x):
    out = self.fc1(x) # input x przechodzi przez pierwszą warstwę
    out = F.relu(out) # stosujemy funkcje aktywacji do pierwszej warsty
    out = self.fc2(out) # output pierwszej warsty przechodzi przez drugą warstę
    if not self.training:
      out = F.softmax(out, dim=1) # jeśli sieć nie jest w trybie trenowania to output drugiej warstwy przechodzi przez funkcje aktywacji softmax
    return out

In [ ]:
model = Model(64,10)

# Trening

## Funkcje błędu

Funkcje błędu dostępne są w module `torch.nn`

Przykład

In [ ]:
ce_loss = nn.CrossEntropyLoss() # klasyfikacja
mse_loss = nn.MSELoss() # regresja

Funkcja będu zawsze zwraca skalar

In [ ]:
ce_loss(Tensor([[0.1,0.1,0.8]]), Tensor([[0,0,1]]))

tensor(0.6897)

In [ ]:
mse_loss(Tensor([[0.1,0.1,0.8]]), Tensor([[0,0,1]]))

tensor(0.0200)

Aby policzyć gradienty za pomoca funkcji błędu wywołujemy `loss.backward()`

In [ ]:
import torch

In [ ]:
x = torch.zeros((1,3), dtype=torch.float32, requires_grad=True)
y = torch.ones((1,3), dtype= torch.float32, requires_grad=True)

In [ ]:
x

tensor([[0., 0., 0.]], requires_grad=True)

In [ ]:
y

tensor([[1., 1., 1.]], requires_grad=True)

In [ ]:
loss =  ce_loss(x,y)

In [ ]:
loss

tensor(3.2958, grad_fn=<DivBackward1>)

In [ ]:
loss.backward()

In [ ]:
x.grad

tensor([[-5.9605e-08, -5.9605e-08, -5.9605e-08]])

Uwaga! Funkcja błęd `CrossEntropyLoss()` w pytorchu ma już zaimplementowany softmax dlatego podczas treningu nie robimy softmaxu na ostatniej warstwie

Więcej funkcji błedu https://neptune.ai/blog/pytorch-loss-functions

## Optymalizatory

Optymalizatory dostepne sa w module `torch.optim`

In [ ]:
from torch import optim

Optymalizatory przy inicjalizacji przyjmują jako argument parametrestry sieci neuronowej (`model.parameters()`). Parametry te będą aktualizowane przez optymalizator podczas treningu

Przykład

In [ ]:
model.parameters

<bound method Module.parameters of Model(
  (fc1): Linear(in_features=64, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=10, bias=True)
)>

In [ ]:
adam = optim.Adam(model.parameters())

### Użycie optymalizatora do aktualiza wag

Optymalizator aktulaizuje parametry sieci neuronej za pomocą funkcji `optimizer.step()` ktora wykorzysuje policzone gradienty wag do zaktualizaowania ich - czyli najpierm musimy policzyć funkcję błedu i wykonać na niej backpropagacje a potem możemy użyć optymalizatora


Przykład

In [ ]:
model = Model(64,10)

adam = optim.Adam(model.parameters())

Wagi warstwy 1 na początku

In [ ]:
model.fc1.weight

Parameter containing:
tensor([[-0.0616, -0.0922, -0.1162,  ..., -0.1025, -0.0513, -0.0688],
        [-0.1053,  0.1241, -0.0617,  ..., -0.0774, -0.0663, -0.1173],
        [ 0.0834, -0.1067,  0.0807,  ...,  0.0043, -0.0565,  0.0366],
        ...,
        [ 0.1230,  0.0417, -0.0150,  ..., -0.0464,  0.1072, -0.0865],
        [ 0.1151,  0.0999, -0.0012,  ..., -0.1156,  0.0622, -0.0384],
        [-0.0100,  0.1086, -0.0601,  ...,  0.1103, -0.0711, -0.0044]],
       requires_grad=True)

Predykcja (100 losowych danych)

In [ ]:
y = model(torch.rand(100,64))

Policzenie funkcji błędu

In [ ]:
loss = ce_loss(y, torch.ones(100,10))

Propagacja błedu po warstawch sieci (liczenie gradientów)

In [ ]:
loss.backward()

Gradient wag

In [ ]:
model.fc1.weight.grad

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0002,  0.0003,  0.0003,  ...,  0.0002,  0.0002,  0.0001],
        [ 0.0098,  0.0101,  0.0105,  ...,  0.0097,  0.0094,  0.0110],
        ...,
        [-0.0101, -0.0107, -0.0087,  ..., -0.0087, -0.0092, -0.0094],
        [ 0.0091,  0.0107,  0.0082,  ...,  0.0073,  0.0090,  0.0086],
        [ 0.0107,  0.0114,  0.0105,  ...,  0.0100,  0.0098,  0.0101]])

Użycie optymalizatora do zaktualizowania wag na podstawie gradientu

In [ ]:
adam.step()

Nowe wartości wag

In [ ]:
model.fc1.weight

Parameter containing:
tensor([[-0.0616, -0.0922, -0.1162,  ..., -0.1025, -0.0513, -0.0688],
        [-0.1063,  0.1231, -0.0627,  ..., -0.0784, -0.0673, -0.1183],
        [ 0.0824, -0.1077,  0.0797,  ...,  0.0033, -0.0575,  0.0356],
        ...,
        [ 0.1240,  0.0427, -0.0140,  ..., -0.0454,  0.1082, -0.0855],
        [ 0.1141,  0.0989, -0.0022,  ..., -0.1166,  0.0612, -0.0394],
        [-0.0110,  0.1076, -0.0611,  ...,  0.1093, -0.0721, -0.0054]],
       requires_grad=True)

### Czyszczenie gradientów

Po każdym aktualizacji wag przez optymalizaor chcemy wyczyścić gradienty (aby policzyć nowe od początko dla innego zestawu danych). Dokonuje się tego za pomocą funkjci `optimizer.zero_grad()`

Przykład

In [ ]:
model.fc1.weight.grad

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0002,  0.0003,  0.0003,  ...,  0.0002,  0.0002,  0.0001],
        [ 0.0098,  0.0101,  0.0105,  ...,  0.0097,  0.0094,  0.0110],
        ...,
        [-0.0101, -0.0107, -0.0087,  ..., -0.0087, -0.0092, -0.0094],
        [ 0.0091,  0.0107,  0.0082,  ...,  0.0073,  0.0090,  0.0086],
        [ 0.0107,  0.0114,  0.0105,  ...,  0.0100,  0.0098,  0.0101]])

In [ ]:
adam.zero_grad()

In [ ]:
model.fc1.weight.grad

# Zadania

## Dataset

Wczytamy podobne dane - ręcznie zapisane cyfry ale w rozmiarze 64x64

In [ ]:
from sklearn.datasets import fetch_openml

In [ ]:
mnist = fetch_openml("mnist_784")

/usr/local/lib/python3.10/dist-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [ ]:
import numpy as np

In [ ]:
data = mnist.data.to_numpy()
targets = np.vectorize(lambda x: int(x))(mnist.target.to_numpy())

In [ ]:
data.shape

(70000, 784)

In [ ]:
targets.shape

(70000,)

### 1. Podziel dane na treningowe i testowe
Podziel dane w stosunku 80/20 ze stratyfikacją

In [ ]:
train_data, test_data, train_targets, test_targets = train_test_split(data, targets, test_size=0.2, stratify=targets, random_state=42)

In [ ]:
train_data.shape

(56000, 784)

In [ ]:
test_data.shape

(14000, 784)

### 2. Zaimplementuj dataset

Uwaga : piksele w nowym datasie są zapisane jako cyfry z przedziału 0-255 zamiast 0-16 jak poprzednio. Uzględnij to w normalizacji

In [ ]:
from torch.utils.data import Dataset


In [ ]:
class DigitsDataset(Dataset):
    def __init__(self, data, targets):
        self.data = data
        self.targets = targets

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        x = torch.tensor(self.data[idx], dtype=torch.float) / 255.0
        y = torch.tensor(self.targets[idx], dtype=torch.long)
        return x, y

In [ ]:
train_dataset = DigitsDataset(train_data, train_targets)

In [ ]:
test_dataset = DigitsDataset(test_data, test_targets)

Weryfikacja danych

X

In [ ]:
train_dataset[0][0]

tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 

y

In [ ]:
train_dataset[0][1]

tensor(0)

### 3. Zaimplementuj dataloader

In [ ]:
from torch.utils.data import DataLoader

In [ ]:
train_dataset = DigitsDataset(train_data, train_targets)
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [ ]:
test_dataset = DigitsDataset(test_data, test_targets)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
from torch import nn
import torch.nn.functional as F


## Sieć Neuronowa

### 1. Zaimplementuj sieć neuronową:

Sieć powinna mieć 2 warswy ukryte - pierwsza warstwa 200 neuronów, druga 100

In [ ]:
class Model(nn.Module):
    def __init__(self, input_size, num_classes):
        super(Model, self).__init__()
        self.fc1 = nn.Linear(input_size, 200)
        self.fc2 = nn.Linear(200, 100)
        self.fc3 = nn.Linear(100, num_classes)

    def forward(self, x):
        out = F.relu(self.fc1(x))
        out = F.relu(self.fc2(out))
        out = self.fc3(out)
        if not self.training:
            out = F.softmax(out, dim=1)  # jeśli sieć nie jest w trybie trenowania, to wynik drugiej warstwy przechodzi przez funkcję aktywacji softmax
        return out

## Trening

In [ ]:
from torch import optim
import torch

### 1. Zainicjalizuj model, funkcje błędu, i optymalizator

Funkcja błędu - entropia krzyżowa

Optymalizator - Adam

In [ ]:
input_size = 784
num_classes = 10

model = Model(input_size, num_classes)

loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

In [ ]:
# from matplotlib import test


train_losses = []
test_losses = []

for epoch in range(10):

  epoch_train_loss = 0.
  epoch_test_loss = 0.


  for data in train_dataloader:
    inputs, labels = data

    #wyzeruj gradienty
    optimizer.zero_grad()

    #policz predykcje modelu (może być potrzebna konwersja danych na float - inputs.float())
    inputs = inputs.float()
    outputs = model(inputs)

    #policz funkcje błędu
    loss = loss_function(outputs, labels)

    #policz gradienty
    loss.backward()

    #zaktualizuj wagi modelu korzystając z optymalizatora
    optimizer.step()

    epoch_train_loss += loss.item()

  with torch.no_grad(): # dla danych testowych nie liczymy gradientów
    for test_data in test_dataloader:
      inputs, labels = test_data

      #policz predykcje modelu (może być potrzebna konwersja danych na float - inputs.float())
      inputs = inputs.float()
      outputs = model(inputs)

      #policz funkcje błędu
      loss = loss_function(outputs, labels)

      epoch_test_loss += loss.item()


  print(f"Epoch {epoch} train loss {epoch_train_loss/len(train_dataloader)}, test_loss {epoch_test_loss/len(test_dataloader)}")
  train_losses.append(epoch_train_loss/len(train_dataloader))
  test_losses.append(epoch_test_loss/len(test_dataloader))

Epoch 0 train loss 0.2601338381144617, test_loss 0.12602457675559778
Epoch 1 train loss 0.09948333137482404, test_loss 0.09255381566302247
Epoch 2 train loss 0.0673732830456325, test_loss 0.08415596651274299
Epoch 3 train loss 0.04925346132793597, test_loss 0.07771022646163259
Epoch 4 train loss 0.037631547379240925, test_loss 0.09172952587604119
Epoch 5 train loss 0.03048379639602042, test_loss 0.08447605388640167
Epoch 6 train loss 0.02586551738741816, test_loss 0.08141037152252042
Epoch 7 train loss 0.021044357922360566, test_loss 0.07887975427015237
Epoch 8 train loss 0.019513308526637567, test_loss 0.09858358068208353
Epoch 9 train loss 0.016517274462492553, test_loss 0.0865987335503105


In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(list(range(10)),train_losses)
plt.plot(list(range(10)),test_losses)